# RedPanda Examples

In [1]:
import pandas, redpanda, sqlalchemy, sys
print "RedPanda version:", redpanda.__version__
print "Python version:", sys.version

RedPanda version: 0.2.3
Python version: 2.7.12 (default, Jun 29 2016, 08:57:23) 
[GCC 5.3.0]


## Opening a session

Open a `RedPanda` session to begin exploring data:

In [2]:
engine = redpanda.create_engine("sqlite://")
session = redpanda.Session(bind=engine)

Note that this is syntactic sugar for SQLAlchemy calls:

In [3]:
engine = sqlalchemy.create_engine("sqlite://")
sessionmaker = sqlalchemy.orm.sessionmaker(class_=redpanda.orm.Session,
                                           query_cls=redpanda.orm.Query)
session = sessionmaker(bind=engine)

## Creating An Example Model

Define your models using the declarative API. `RedPanda` defines one such model in the `example` module:

In [4]:
import sqlalchemy.ext.declarative
Base = sqlalchemy.ext.declarative.declarative_base()
class Widget(Base):
    id        = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    timestamp = sqlalchemy.Column(sqlalchemy.DateTime)
    name      = sqlalchemy.Column(sqlalchemy.String)
    kind      = sqlalchemy.Column(sqlalchemy.String)
    units     = sqlalchemy.Column(sqlalchemy.Integer)

    __read_sql__  = {"index_col": ["timestamp"], "parse_dates": ["timestamp"]}
    __tablename__ = "widgets"

Defining the class attribute `__read_sql__` will configure default arguments passed to `pandas.read_sql()`. 

In this case we are configuring the `widgets` table to use the `timestamp` column as an index and that we wish for `pandas` to parse this column as a `datetime` object.

In [5]:
from redpanda.example import create_widgets
create_widgets(session)

## Querying Data

Use the `frame()` method of a query to return the results as a `DataFrame`:

In [6]:
session.query(Widget).frame().head()

,id,name,kind,units
timestamp,,,,
2016-01-02 04:24:00,1,approach,bopper,93
2016-01-05 20:23:00,2,blots,bopper,70
2016-01-07 23:31:00,3,victim,buzzer,53
2016-01-16 18:52:00,4,hammer,bopper,82
2016-01-20 17:08:00,5,fares,bopper,87


### Querying with Filters

Use SQLAlchemy filters to refine the query:

In [7]:
session.query(Widget).filter(Widget.timestamp.between("2016-11-01", "2016-11-30")).frame()

,id,name,kind,units
timestamp,,,,
2016-11-06 14:53:00,61,interfaces,fizzer,37
2016-11-12 02:39:00,62,stakes,bopper,40
2016-11-13 09:07:00,63,hums,buzzer,63
2016-11-17 04:16:00,64,residue,bopper,39
2016-11-24 12:41:00,65,tasks,bopper,17


### Overriding Defaults

Pass arguments to `pandas.read_sql()` into the `frame()` method:

In [8]:
session.query(Widget).frame(index_col="id").head()

,timestamp,name,kind,units
id,,,,
1,2016-01-02 04:24:00,approach,bopper,93
2,2016-01-05 20:23:00,blots,bopper,70
3,2016-01-07 23:31:00,victim,buzzer,53
4,2016-01-16 18:52:00,hammer,bopper,82
5,2016-01-20 17:08:00,fares,bopper,87


## Writing DataFrame to Session

Use the `add_dataframe()` method of the `RedPanda` session to place `DataFrame` rows into the session as models:

In [9]:
from datetime import datetime
dfdata = {
    "kind": "newkind", 
    "name": "newname", 
    "units": 1000, 
    "timestamp": datetime.utcnow()}
addme = pandas.DataFrame(dfdata, index=[0])
addme

,kind,name,timestamp,units
0,newkind,newname,2016-07-13 16:34:19.376921,1000


In [10]:
session.add_dataframe(Widget, addme)
session.commit()
session.query(Widget).filter(Widget.kind=="newkind").frame()

,id,name,kind,units
timestamp,,,,
2016-07-13 16:34:19.376921,76,newname,newkind,1000
